In [ ]:
import json, psycopg2
from psycopg2 import extras

import os
from dotenv import load_dotenv

load_dotenv()

conn = psycopg2.connect(
    dbname=os.getenv("DB_NAME", "postgres"),
    user=os.getenv("DB_USER", "postgres"), 
    password=os.getenv("DB_PASSWORD"),
    host=os.getenv("DB_HOST"),
    port=int(os.getenv("DB_PORT", "5432")),
    sslmode="require",
)
conn.autocommit = True

DDL = """
CREATE TABLE IF NOT EXISTS public.clinical_notes (appointment_id TEXT PRIMARY KEY);
ALTER TABLE public.clinical_notes
  ADD COLUMN IF NOT EXISTS patient_id TEXT,
  ADD COLUMN IF NOT EXISTS user_id TEXT,
  ADD COLUMN IF NOT EXISTS description TEXT,
  ADD COLUMN IF NOT EXISTS created_at TEXT,
  ADD COLUMN IF NOT EXISTS note TEXT,
  ADD COLUMN IF NOT EXISTS date_of_appt TEXT,
  ADD COLUMN IF NOT EXISTS date_of_birth TEXT,
  ADD COLUMN IF NOT EXISTS status TEXT,
  ADD COLUMN IF NOT EXISTS mrn TEXT,
  ADD COLUMN IF NOT EXISTS coding_information TEXT,
  ADD COLUMN IF NOT EXISTS los_code TEXT,
  ADD COLUMN IF NOT EXISTS time_spent NUMERIC,
  ADD COLUMN IF NOT EXISTS time_based_code TEXT,
  ADD COLUMN IF NOT EXISTS icd_10_codes JSONB,
  ADD COLUMN IF NOT EXISTS orders TEXT,
  ADD COLUMN IF NOT EXISTS import_type TEXT,
  ADD COLUMN IF NOT EXISTS appt_time TEXT,
  ADD COLUMN IF NOT EXISTS note_upload_status TEXT,
  ADD COLUMN IF NOT EXISTS patient_data JSONB,
  ADD COLUMN IF NOT EXISTS diarized_transcript TEXT,
  ADD COLUMN IF NOT EXISTS diarized_transcript_csv_url TEXT,
  ADD COLUMN IF NOT EXISTS patient_type TEXT,
  ADD COLUMN IF NOT EXISTS chart_history JSONB,
  ADD COLUMN IF NOT EXISTS surgery TEXT,
  ADD COLUMN IF NOT EXISTS hcc_codes JSONB,
  ADD COLUMN IF NOT EXISTS cpt_codes TEXT,
  ADD COLUMN IF NOT EXISTS hpi TEXT,
  ADD COLUMN IF NOT EXISTS interval_history TEXT,
  ADD COLUMN IF NOT EXISTS assessment TEXT,
  ADD COLUMN IF NOT EXISTS plan TEXT,
  ADD COLUMN IF NOT EXISTS icd10_codes_v2 JSONB,
  ADD COLUMN IF NOT EXISTS ehr_upload_status TEXT,
  ADD COLUMN IF NOT EXISTS mdm_suggested JSONB,
  ADD COLUMN IF NOT EXISTS mdm_selected JSONB,
  ADD COLUMN IF NOT EXISTS macros JSONB,
  ADD COLUMN IF NOT EXISTS patient_summary TEXT,
  ADD COLUMN IF NOT EXISTS e_and_m TEXT,
  ADD COLUMN IF NOT EXISTS orders_v2 JSONB;
"""
with conn.cursor() as cur:
    cur.execute(DDL)


In [ ]:
# !python3.10 -m pip install psycopg2-binary
import json
import psycopg2 # type: ignore[reportMissingImports]
from psycopg2 import extras  # type: ignore[reportMissingImports]

conn = psycopg2.connect(
    dbname=os.getenv("POSTGRES_DB", "postgres"),
    user=os.getenv("POSTGRES_USER", "postgres"), 
    password=os.getenv("POSTGRES_PASSWORD"),
    host=os.getenv("POSTGRES_HOST"),
    port=int(os.getenv("POSTGRES_PORT", "5432")),
)

# Columns in table order (include everything you expect)
COLS = [
    "appointment_id","patient_id","user_id","mrn","description","status",
    "date_of_appt","created_at","los_code","time_spent","note",
    "coding_information","orders","diarized_transcript","patient_type",
    "cpt_codes","hpi","plan","icd10_codes_v2","chart_history",
    "mdm_suggested","mdm_selected","date_of_birth","time_based_code",
    "icd_10_codes","import_type","appt_time","note_upload_status",
    "patient_data","diarized_transcript_csv_url","surgery","hcc_codes",
    "interval_history","assessment","ehr_upload_status","macros",
    "patient_summary","e_and_m","orders_v2"
]

# Which columns are JSONB in PostgreSQL
JSONB_COLS = {
    "icd10_codes_v2","chart_history","mdm_suggested","mdm_selected",
    "icd_10_codes","patient_data","hcc_codes","macros","orders_v2"
}

def to_jsonb(v):
    if v is None: return None
    if isinstance(v,(dict,list)): return extras.Json(v)
    if isinstance(v,str):
        s = v.strip()
        try:
            return extras.Json(json.loads(s))  # parse stringified JSON
        except Exception:
            return extras.Json(s)              # store as JSON string
    return extras.Json(v)

with conn, conn.cursor() as cur:
    data = json.load(open("postgres.json","r"))

    rows = []
    for r in data:
        rec = []
        for c in COLS:
            val = r.get(c)
            if c in JSONB_COLS:
                rec.append(to_jsonb(val))
            elif c == "time_spent":
                rec.append(float(val) if val not in (None,"","None") else None)
            else:
                rec.append(val)
        rows.append(tuple(rec))

    cols_sql = ", ".join([f'"{c}"' for c in COLS])
    update_sql = ", ".join([f'"{c}"=EXCLUDED."{c}"' for c in COLS if c != "appointment_id"])

    extras.execute_values(
        cur,
        f"""
        INSERT INTO public.clinical_notes ({cols_sql})
        VALUES %s
        ON CONFLICT ("appointment_id") DO UPDATE SET
        {update_sql};
        """,
        rows
    )